In [1]:
pip install dash

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip3.11 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install --upgrade nbformat

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip3.11 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install dash_design_kit

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for dash_design_kit from https://files.pythonhosted.org/packages/6a/ac/edacc3a663b8a20f88a00f7d0962c8afb34db16ac2320cb5cd6d816eb672/dash_design_kit-0.0.1-py2.py3-none-any.whl.metadata

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip3.11 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [18]:
pip install nbformat

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for nbformat from https://files.pythonhosted.org/packages/a9/82/0340caa499416c78e5d8f5f05947ae4bc3cba53c9f038ab6e9ed964e22f1/nbformat-5.10.4-py3-none-any.whl.metadata
  Using cached nbformat-5.10.4-py3-none-any.whl.metadata (3.6 kB)
Using cached nbformat-5.10.4-py3-none-any.whl (78 kB)

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip3.11 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [13]:
from matplotlib import pyplot as plt
import matplotlib as mpl
from dotenv import load_dotenv
import pandas as pd 
from dash import Dash, html, dash_table, dcc
import plotly.express as px
import boto3
import io
import os

load_dotenv()

True

In [3]:
bucket = os.getenv('AWS_BUCKET_NAME')
access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
secret_key = os.getenv('AWS_PRIVATE_KEY')

s3_client = boto3.client(
        's3',
        aws_access_key_id=access_key_id,
        aws_secret_access_key=secret_key)

read_response = s3_client.get_object(Bucket=bucket, Key='EPL 2024-2025 xG.csv')

xG_df = pd.read_csv(io.BytesIO(read_response['Body'].read()))
xG_df['xGD cumulative sum'] = xG_df['xGD_sum_over_gameweek']

top5 = xG_df[xG_df['gameweek']==xG_df['gameweek'].max()].sort_values(by='xGD cumulative sum', ascending=False).head(5)

top5_xG_teams = top5['team'].unique()
top_5_xGD = xG_df[xG_df['team'].isin(top5_xG_teams)]

bottom5 = xG_df[xG_df['gameweek']==xG_df['gameweek'].max()].sort_values(by='xGD cumulative sum', ascending=True).head(5)

bottom5_xG_teams = bottom5['team'].unique()
bottom_5_xGD = xG_df[xG_df['team'].isin(bottom5_xG_teams)]

In [7]:
app = Dash()

In [5]:
plt.figure(figsize=(12, 8))

<Figure size 1200x800 with 0 Axes>

<Figure size 1200x800 with 0 Axes>

In [6]:
fig = px.line(top_5_xGD, x='gameweek', y='xGD cumulative sum', color='team')

fig.show()

In [ ]:
grouped_xGD_top5 = top_5_xGD[top_5_xGD['gameweek']==top_5_xGD['gameweek'].max()].sort_values(by='xGD cumulative sum', ascending=False).drop(columns=['gameweek', 'xG', 'xGA', 'xGD', 'date', 'xGD_sum_over_gameweek'])
grouped_xGD_top5.head()

,team,xGD cumulative sum
237,Liverpool,12.6
226,Tottenham,8.5
224,Manchester City,8.4
232,Chelsea,6.3
221,Arsenal,6.2


In [37]:
app.layout = html.Div([
    dcc.Graph(
        id='top 5 xGD teams',
        figure=fig
    ),
    dash_table.DataTable(id='xGD table', 
                         columns=[{'name': i, 'id': i} for i in grouped_xGD_top5.columns], 
                         data=grouped_xGD_top5.to_dict('records'),
                         style_cell=dict(textAlign='left'),
                         style_header=dict(backgroundColor='paleturquoise'),
                         style_data=dict(backgroundColor='lavender'))
])

In [38]:
if __name__ == '__main__':
    app.run()